## Соревнование Thousand Facial Landmarks по Computer Vision в MADE
Студент: Алексей Ярошенко

**Что сработало:**
- Т.к. модели обучены на imagenet, взял нормировочные константы оттуда.
-  Увеличил CROP_SIZE до 224, как в imagenet. К тому же, подумал, что чем выше разрешение, тем можно точнее попасть в нужный пиксель.
- Увеличил размер трейна с 80 до 97%. Для адекватной валидации хватало 3% (11 000+ картинок), датасет  относительно большой.
- Модели учились чутка нестабильно, по разному. Долго с этим возился. В конце концов, пришел к тому, что использовал sсheduler CosineAnnealingWarmRestarts, чтобы менять lr по синусоиде от максимума до нуля, каждый раз удваивая период после рестарта. С ним модели начали лучше и стабильнее сходиться. В статьях пишут, что с ним больше вероятность найти лучший локальный минимум. У меня работал лучше чем ReduceLROnPlateau.
- Учил модели адамом, дообучал на SGD. Прочел где-то, попробовал так делать, и иногда получалось небольшое повышение метрики выжать. Хотя возможно, с SGD сеть медленнее училась и на маленьком размере батча просто не успевала переобучиться.
- Делал рестарты. Т.е. если видел, что модель не сходится/переобучается и откатывался к последнему чекпойнту и менял что-то: оптимайзер, LR, перезапускал scheduler.
- Добавил регуляризацию. Небольшой weight_decay делал модель устойчивее на самых последних этапах, когда сеточка начинает переобучаться. Скор на валидации от этого не рос, но на паблике это было в плюс.
- Протестировал много предобученных моделей из torchvision. Лучше всего зашли densenet
- Я понадеялся, что ошибки хотя бы на некоторых точках нормально распределены и если я наставлю несколько точек на плоскости, то центр будет лучше предсказывать точку. Скачал топ своих лучших сабмитов для разных архитектур/моделей, усреднил предсказания. К тому же, скорее всего, это сделало модель стабильнее и дало еще небольшой буст после конца соревнования, когда открылся private leaderboard. На паблике я за 2 часа до конца стал вторым, на привате снова выкинуло на 1-е место.
- Усреднил топ сабмитов со взвешиванием: у лучшего скора больший вес.
- Логировал результат трейна и теста каждой эпохи в телеграм, чтобы, если, к примеру случайно проснулся ночью, можно при необходимости сделать рестарт обучения с новыми параметрами. Из кода убрал, ибо он особого смысла не несет.

**Что не помогло:**
- Замораживать слои. Дообучать разные слои с разным lr. Дообучать по очереди. Быстрее становилось, лучше - нет. Нужно, к тому же, за этим следить.
- Подмешивать какой-то еще лосс (к примеру smooth_l1_loss). Наверное, это логично, ибо не представляю, что может быть лучше оптимизации MSE, когда у нас целевая метрика - MSE. Только если как своеобразный регуляризатор, наверное.
- Простые аугментации вроде контраста, насыщенности и т.д. К тому же, данных достаточно, а если накинуть пару аугментаций, все совсем долго считается.
- Поднимать разрешение выше 224. Больших картинок не очень много, а учится сеть неприлично дольше. Батчи становятся все меньше, ибо памяти не хватает. Возможно, и дало бы скор лучше, но я бы его просто не дождался.

**Чего не делал:**
- Не очень удобных в написании аугментаций к примеру, с поворотом. 
- Не чистил выбросы. Наверное, стоило бы. Но я подумал, что раз выбросы есть и в тесте, и трейне, пускай модель хоть что-то из них выучит. Но тут без понятия.
- Не использовал сторонних библиотек и архитектур кроме torchvision.models.

![Submit screeshot](submit_screenshot.png)

P.S.: ансамблирование с описанием модели для каждого сабмита, который я использовал в ансамбле, в самом низу ноутбука.

In [1]:
import os
import gc
import pickle
import sys

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import tqdm
from torch.nn import functional as fnn
from torch.utils import data
from torchvision import transforms
from IPython import display
import matplotlib.pyplot as plt

from hack_utils import NUM_PTS, CROP_SIZE
from hack_utils import ScaleMinSideToSize, CropCenter, TransformByKeys
from hack_utils import ThousandLandmarksDataset
from hack_utils import restore_landmarks_batch, create_submission

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
DATA_PATH = 'data/'
MODEL_NAME = 'densenet169_anealing_b16_train097_size224_norm_imgnet'
GPU = True
BATCH_SIZE = 16
EPOCHS = 30
CROP_SIZE = 224
NORM_MEAN = [0.485, 0.456, 0.406]
NORM_STD = [0.229, 0.224, 0.225]

In [3]:
def train(model, loader, loss_fn, optimizer, device):
    model.train()
    train_loss = []
    for batch in tqdm.tqdm(loader, total=len(loader), desc="training..."):
        images = batch["image"].to(device)  # B x 3 x CROP_SIZE x CROP_SIZE
        landmarks = batch["landmarks"]  # B x (2 * NUM_PTS)

        pred_landmarks = model(images).cpu()  # B x (2 * NUM_PTS)
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")        
        train_loss.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()  

    return np.mean(train_loss)

# для CosineAnnealingWarmRestarts
def train_anealing(model, loader, loss_fn, optimizer, scheduler, epoch, device):
    model.train()
    iters = len(loader)
    i = 0
    train_loss = []
    for batch in tqdm.tqdm(loader, total=len(loader), desc="training...", position=0, leave=True):
        images = batch["image"].to(device)  # B x 3 x CROP_SIZE x CROP_SIZE
        landmarks = batch["landmarks"]  # B x (2 * NUM_PTS)

        pred_landmarks = model(images).cpu()  # B x (2 * NUM_PTS)
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        train_loss.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step(epoch + i / iters)
        i += 1

    return np.mean(train_loss)

def validate(model, loader, loss_fn, device):
    model.eval()
    val_loss = []
    for batch in tqdm.tqdm(loader, total=len(loader), desc="validation..."):
        images = batch["image"].to(device)
        landmarks = batch["landmarks"]

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        val_loss.append(loss.item())

    return np.mean(val_loss)


def predict(model, loader, device):
    model.eval()
    predictions = np.zeros((len(loader.dataset), NUM_PTS, 2))
    for i, batch in enumerate(tqdm.tqdm(loader, total=len(loader), desc="test prediction...")):
        images = batch["image"].to(device)

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
        pred_landmarks = pred_landmarks.numpy().reshape((len(pred_landmarks), NUM_PTS, 2))  # B x NUM_PTS x 2

        fs = batch["scale_coef"].numpy()  # B
        margins_x = batch["crop_margin_x"].numpy()  # B
        margins_y = batch["crop_margin_y"].numpy()  # B
        prediction = restore_landmarks_batch(pred_landmarks, fs, margins_x, margins_y)  # B x NUM_PTS x 2
        predictions[i * loader.batch_size: (i + 1) * loader.batch_size] = prediction

    return predictions

### Подготовка датасета

In [4]:
train_transforms = transforms.Compose([
    ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
    CropCenter(CROP_SIZE),
    TransformByKeys(transforms.ToPILImage(), ("image",)),
    TransformByKeys(transforms.ToTensor(), ("image",)),
    TransformByKeys(transforms.Normalize(mean=NORM_MEAN, std=NORM_STD), ("image",)),
])

test_transforms = transforms.Compose([
    ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
    CropCenter(CROP_SIZE),
    TransformByKeys(transforms.ToPILImage(), ("image",)),
    TransformByKeys(transforms.ToTensor(), ("image",)),
    TransformByKeys(transforms.Normalize(mean=NORM_MEAN, std=NORM_STD), ("image",)),
])

train_dataset = ThousandLandmarksDataset(os.path.join(DATA_PATH, 'train'), train_transforms, split="train")
val_dataset = ThousandLandmarksDataset(os.path.join(DATA_PATH, 'train'), test_transforms, split="val")

382112it [06:07, 1040.19it/s]
393931it [00:12, 31934.70it/s] 


In [5]:
train_dataloader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=7, pin_memory=True,
                                   shuffle=True, drop_last=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=7, pin_memory=True,
                                 shuffle=False, drop_last=False)

### Подготовка модели

In [6]:
device = torch.device("cuda: 0") if GPU else torch.device("cpu")
model = models.densenet169(pretrained=True)

model.classifier = nn.Linear(model.classifier.in_features, 2 * NUM_PTS, bias=True)
model.to(device)

# Один из оптимайзеров закомментирован, т.к. часто использовал оба. Сначала Adam, потом SGD
# optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum=0.9, nesterov=True)
optimizer = optim.Adam(model.parameters(), lr=1e-3, amsgrad=True, weight_decay=0.00001)

# Меняем LR от максимума до нуля по синусоиде за 1 эпоху. 
# Потом повторяем, каждый раз с периодом в 2 раза больше
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, 
    T_0=len(train_dataloader), 
    T_mult=2
)
loss_fn = fnn.mse_loss

Для рестарта, чтобы дальше обучать модель

In [7]:
# with open(f"{MODEL_NAME}_best.pth", "rb") as fp:
#     best_state_dict = torch.load(fp, map_location="cpu")
#     model.load_state_dict(best_state_dict)

### Обучение модели

In [8]:
best_val_loss = validate(model, val_dataloader, loss_fn, device=device)
print("Start val loss: {:5.4}".format(best_val_loss))

torch.cuda.empty_cache()
for epoch in range(EPOCHS):
    train_loss = train_anealing(model, train_dataloader, loss_fn, optimizer, scheduler, epoch, device=device)
    val_loss = validate(model, val_dataloader, loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.4}\tval loss: {:5.4}".format(epoch, train_loss, val_loss))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        with open(f"{MODEL_NAME}_best.pth", "wb") as fp:
            torch.save(model.state_dict(), fp)

training...:   0%|          | 0/23881 [00:00<?, ?it/s]

Start val loss: 1.36e+04


validation...: 100%|██████████| 739/739 [00:56<00:00, 13.10it/s]

Epoch # 0:	train loss: 44.95	val loss: 7.868


### Делаем сабмит

In [9]:
test_dataset = ThousandLandmarksDataset(os.path.join(DATA_PATH, 'test'), test_transforms, split="test")
test_dataloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=8, pin_memory=True,
                                  shuffle=False, drop_last=False)

with open(f"{MODEL_NAME}_best.pth", "rb") as fp:
    best_state_dict = torch.load(fp, map_location="cpu")
    model.load_state_dict(best_state_dict)

test_predictions = predict(model, test_dataloader, device)
with open(f"{MODEL_NAME}_test_predictions.pkl", "wb") as fp:
    pickle.dump({"image_names": test_dataset.image_names,
                 "landmarks": test_predictions}, fp)


del test_dataset, test_dataloader
gc.collect()

create_submission(DATA_PATH, test_predictions, f"{MODEL_NAME}_submit.csv")

99820it [00:00, 765764.15it/s]
test prediction...: 100%|██████████| 6239/6239 [07:57<00:00, 13.08it/s]


### Ансамблирование

In [ ]:
import pandas as pd
import numpy as np
from scipy.special import softmax

In [ ]:
# Файл сабмита на kaggle: densenet201_anealing_b16_train095_size224_norm_imgnet_submit.csv
# Архитектура: densenet201
# Размер батча: 16
# Scheduler: CosineAnnealingWarmRestarts (T_mult=2)
# Размер обучающей выборки: 0.95
# Размер картинки: 224х224
# Результат на паблике: 8.04
df_1 = pd.read_csv('submit_804.csv')

# Файл сабмита на kaggle: resnext101_32x8d_anealing_b16_train095_size224_norm_imgnet_submit.csv
# Архитектура: resnext101_32x8d
# Размер батча: 16
# Scheduler: CosineAnnealingWarmRestarts (T_mult=2)
# Размер обучающей выборки: 0.95
# Размер картинки: 224х224
# Результат на паблике: 8.14
df_2 = pd.read_csv('submit_814.csv')

# Файл сабмита на kaggle: resnext50_32x4d_anealing_b32_train097_size224_norm_imgnet_submit.csv
# Архитектура: resnext50_32x4d
# Размер батча: 32
# Scheduler: CosineAnnealingWarmRestarts (T_mult=2)
# Размер обучающей выборки: 0.97
# Размер картинки: 224х224
# Результат на паблике: 8.22
df_3 = pd.read_csv('submit_822.csv')

# Файл сабмита на kaggle: resnet34_2_anealing_b64_train097_size224_norm_imgnet_submit.csv
# Архитектура: resnet34
# Размер батча: 64
# Scheduler: CosineAnnealingWarmRestarts (T_mult=2)
# Размер обучающей выборки: 0.97
# Размер картинки: 224х224
# Результат на паблике: 8.32
df_4 = pd.read_csv('submit_832.csv')

# Файл сабмита на kaggle: densenet169_anealing_b16_train097_size224_norm_imgnet_submit.csv
# Архитектура: densenet169
# Размер батча: 16
# Scheduler: CosineAnnealingWarmRestarts (T_mult=2)
# Размер обучающей выборки: 0.97
# Размер картинки: 224х224
# Результат на паблике: 8.33
df_5 = pd.read_csv('submit_833.csv')

# Файл сабмита на kaggle: densenet169_pipeline5_anealing_b32_train09_size224_norm_imgnet_submit.csv
# Архитектура: densenet169
# Размер батча: 32
# Scheduler: CosineAnnealingWarmRestarts (T_mult=2)
# Размер обучающей выборки: 0.9
# Размер картинки: 224х224
# Результат на паблике: 8.34
df_6 = pd.read_csv('submit_834.csv')

# Файл сабмита на kaggle: resnet34_sgd_annealing_b32_lr1e3_train095_size256_norm_imgnet_submit.csv
# Архитектура: resnet34
# Размер батча: 32
# Scheduler: CosineAnnealingWarmRestarts (T_mult=2)
# Learning rate: 1e-3
# Размер обучающей выборки: 0.95
# Размер картинки: 256х256
# Результат на паблике: 8.47
df_7 = pd.read_csv('submit_847.csv')

# Файл сабмита на kaggle: resnext50_32x4d_anealing_b32_train097_size224_norm_imgnet_submit.csv
# Архитектура: resnext50_32x4d
# Размер батча: 32
# Scheduler: CosineAnnealingWarmRestarts (T_mult=2)
# Размер обучающей выборки: 0.97
# Размер картинки: 224х224
# Результат на паблике: 8.49
df_8 = pd.read_csv('submit_849.csv')

# Файл сабмита на kaggle: densenet121_anealing_b32_train097_size224_norm_imgnet_submit.csv
# Архитектура: densenet121
# Размер батча: 32
# Размер обучающей выборки: 0.97
# Размер картинки: 224х224
# Результат на паблике: 8.51
df_9 = pd.read_csv('submit_851.csv')

# Файл сабмита на kaggle: resnet50_3_nofreeze_steps_b128_lr3e4_train09_size224_norm_imgnet_submit.csv
# Архитектура: resnet50
# Размер батча: 128
# Размер обучающей выборки: 0.9
# Размер картинки: 224х224
# Обучалась по частям: сначала классификатом, потом по шагам размораживались слои и снижался Learning rate
# Результат на паблике: 8.57
df_10 = pd.read_csv('submit_857.csv')

# Файл сабмита на kaggle: densenet121_anealing_b32_lr1e3_train09_size224_norm_imgnet_submit.csv
# Архитектура: densenet121
# Размер батча: 32
# Scheduler: CosineAnnealingWarmRestarts (T_mult=2)
# Learning rate: 1e-3
# Размер обучающей выборки: 0.9
# Размер картинки: 224х224
# Результат на паблике: 8.59
df_11 = pd.read_csv('submit_859.csv')

# Файл сабмита на kaggle: resnext50_32x4d_b32_lr3e4_train09_size224_norm_imgnet_submit.csv
# Архитектура: resnext50_32x4d
# Размер батча: 32
# Learning rate: 3e-4
# Размер обучающей выборки: 0.9
# Размер картинки: 224х224
# Результат на паблике: 8.74
df_12 = pd.read_csv('submit_874.csv')

In [ ]:
dfs = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11, df_12]

# Перевзвесим сабмиты, чтобы лучшие сабмиты получили больший вес
w = softmax(np.arange(len(dfs) + 1, 1, -1) / 10)

df_result = dfs[0].copy()
df_result.iloc[:, 1:] = df_result.iloc[:, 1:] * w[0]
for i, df in enumerate(dfs[1:]):
    df_result.iloc[:, 1:] += df.iloc[:, 1:] * w[i + 1]
    
df_result.iloc[:, 1:] = df_result.iloc[:, 1:].round().astype(int)

df_result.to_csv('submit_ensemble.csv', index=False)